In [ ]:
from qdrant_client import QdrantClient
from vllm import LLM
from vllm.sampling_params import SamplingParams
from qdrant_client.models import Distance, VectorParams
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv
import os
from qdrant_client.models import PointStruct
from groq import Groq
import gradio as gr

In [ ]:
load_dotenv()

In [ ]:
from huggingface_hub import login

login(token=os.getenv("HUGGING_FACE_TOKE"))
groq_api = Groq(api_key = os.getenv("GROQ_API"))
                
model_name= "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}

embeddings = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
llm=LLM(
    model="mistral-community/pixtral-12b-240910",
    tokenizer_mode="mistral",
    max_model_len=5000
)

In [ ]:
qdrant_client = QdrantClient(":memory:")

In [ ]:
def generate_data(image_url, prompt = "Extract the information from the image"):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": image_url}}
            ]
        }
    ]

    outputs = llm.chat(
        messages,
        sampling_params=SamplingParams(max_tokens=8192)
    )

    return outputs[0].outputs[0].text


In [ ]:
def generate_embeddings(data_text):
  return embeddings.encode(data_text)

In [ ]:
def initialize_qdrant(length: int):
    vector_size = length
    # Define the vectors configuration
    vector_params = VectorParams(
        size=vector_size,                # Size of the vectors
        distance=Distance.COSINE         # Choose distance metric (COSINE, EUCLID, or IP)
    )
    
    # Create the collection with the specified configuration
    qdrant_client.create_collection(
        collection_name="CHATBOT",
        vectors_config=vector_params  # Specify vector configuration
    )

In [ ]:
 def prepare_embeddings(data, batch_size = 1):
     data = data.split('\n')
     
     for it in range(len(data)):
         data[it] = [data[it]]
        
         
     total_items = len(data)
     batched_data = []
 
     for start in range(0, total_items, batch_size):
         end = min(start + batch_size, total_items)
         batch = [item for item in data[start:end]]  # Extract contexts for this batch
         vectors = generate_embeddings(batch)  # Generate embeddings for the batch

         for i, vector in enumerate(vectors):
             data[start + i].append(vector)  # Append vector to the corresponding item
     return data

In [ ]:
def qdrant_entry(final_data):
    points=[PointStruct( id=i,  vector=final_data[i][2],payload={'raw_context':final_data[i][0] }) for i in range(len(final_data))]
    qdrant_client.upsert(collection_name="CHATBOT", points=points)
    print(qdrant_client.get_collections())




In [ ]:
 def query_qdrant(query, collection_name='CHATBOT', limit=4):
    query_vector=generate_embeddings(query)
    result = qdrant_client.search(
        collection_name="CHATBOT",
        query_vector=query_vector,
        limit=limit,
        with_vectors=False
    )
    search_result=[]
    
    return result

In [ ]:
def respond(question):
    chat_completion = groq_api.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the question asked by user {question} and the context given is {'-----'.join(query_qdrant(question))} answer this question based on the context provided",
            }
                ],
                model="llama-3.1-70b-versatile",
            )

    return chat_completion.choices[0].message.content

In [ ]:
def processing(query):
        answer=respond(query)
        return str(answer)

In [ ]:
if __name__=="__main__":
    choice = input("Enter 'query' for rag query \n 'entry' for rag entry")
    if choice == 'entry':
        url = "https://assets.techrepublic.com/uploads/2017/04/aexcelpowerbi.png"
        
        data = generate_data(url)
        final_data=prepare_embeddings(data,batch_size=10)
    
        initialize_qdrant(final_data[0][1])

        qdrant_entry(final_data)
    else:
        demo = gr.Interface(
        fn=processing,
        inputs=["text"],
        outputs=["text"],
                )
        demo.launch(debug=True)
        